# RAG Complaint Analysis Notebook

- This notebook demonstrates running the full RAG pipeline with the Mistral-7B-Instruct model, retrieving complaint chunks, generating answers, and performing qualitative evaluation in a clean and reusable way.

Import Libraries

In [1]:
import sys
import os
import pandas as pd

# -------------------------
#  FIX PYTHON PATH FOR NOTEBOOK
# -------------------------

# Path to project root (one level above 'notebooks')
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Path to `src` directory
scripts_path = os.path.join(project_root, "src")

# Add both project root and scripts folder to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

if scripts_path not in sys.path:
    sys.path.append(scripts_path)

print("Project root:", project_root)
print("Scripts path:", scripts_path)

# -------------------------
#  IMPORTS
# -------------------------

try:
    from src.rag_pipeline import build_rag_pipeline
    from src.evaluation import run_evaluation, results_to_dataframe, results_to_markdown_table, EVALUATION_QUESTIONS
    from src.vector_store import ComplaintVectorStore
    from src.retriever import build_retriever, ComplaintRetriever
    from src.generator import build_generator, RAGGenerator
    print("Imports successful!")
except ModuleNotFoundError as e:
    print("Import failed:", e)
    raise

Project root: /Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot
Scripts path: /Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/src


/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/week7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports successful!


In [ ]:
vectorstore = ComplaintVectorStore.from_parquet(
    parquet_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/data/raw/complaint_embeddings.parquet",
    index_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/faiss.index",
    meta_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/metadata.json",
    batch_size=5000
)

In [2]:
vectorstore = ComplaintVectorStore.load(
    index_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/faiss.index",
    meta_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/metadata.json"
)

#### Initialize RAG Pipeline\n",
Build retriever + generator with Mistral model

In [3]:
# Pass the actual file paths as strings
retriever = build_retriever(
    index_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/faiss.index",
    meta_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/metadata.json"
)



In [4]:
generator = build_generator(
    llm_model_path="/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"
)

llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

In [5]:
questions = [
    # Credit Card
    "Why do customers report being charged unexpected fees on their credit cards?",
    "What are the common complaints regarding credit card customer service or dispute handling?",
    # Personal Loan
    "Why are some personal loan applications denied, according to customer complaints?",
    "What issues do customers report about personal loan repayment or interest rates?",
    # Savings Account
    "What problems do customers report with opening or managing savings accounts?",
    "Are there complaints about delayed transactions or account access issues in savings accounts?",
    # Money Transfers
    "What are the most frequent complaints customers have about domestic or international money transfers?",
    "Why do customers report delays or failures when making money transfers?"
]


In [ ]:
# --- 3. Semi-automatic evaluation function ---
def evaluate_answer(answer: str, sources: list):
    """
    Automatically assign scores based on overlap with sources.
    - Accuracy: proportion of words in answer that appear in sources
    - Relevance: simple keyword overlap with question (1-5)
    - Completeness: proportion of sources covered
    - Hallucination: Yes if words appear that aren't in any source
    """
    answer_words = set(answer.lower().split())
    source_words = set()
    for s in sources:
        source_words.update(s.lower().split())

    # Accuracy score: % words in answer that are also in sources
    accuracy_ratio = len(answer_words & source_words) / max(1, len(answer_words))
    accuracy = min(5, max(1, round(accuracy_ratio * 5)))

    # Completeness: % of sources contributing at least one word to answer
    covered_sources = sum(1 for s in sources if len(set(s.lower().split()) & answer_words) > 0)
    completeness_ratio = covered_sources / max(1, len(sources))
    completeness = min(5, max(1, round(completeness_ratio * 5)))

    # Relevance: using same heuristic as accuracy (could be improved)
    relevance = accuracy

    # Hallucination: if any word not in sources
    hallucination = "No" if answer_words <= source_words else "Yes"

    return accuracy, relevance, completeness, hallucination

# --- 4. Run evaluation ---
results = []

for q in EVALUATION_QUESTIONS:
    retrieved_chunks = retriever.retrieve(q, k=5)
    answer = generator.generate(q, retrieved_chunks)

    # Prepare source texts for evaluation
    sources = [item['metadata'].get('issue','') + " " + item['metadata'].get('company','') for item in retrieved_chunks]

    accuracy, relevance, completeness, hallucination = evaluate_answer(answer, sources)

    results.append({
        "Question": q,
        "Generated Answer": answer,
        "Retrieved Sources": " | ".join(sources[:2]),
        "Accuracy (1-5)": accuracy,
        "Relevance (1-5)": relevance,
        "Completeness (1-5)": completeness,
        "Hallucination (Yes/No)": hallucination,
        "Comments": ""  # Optional manual notes
    })



# --- 5. Save evaluation results to CSV ---
output_path = f"/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/data/rag_evaluation_results.csv"
df_evaluation.to_csv(output_path, index=False)
print(f"✅ Evaluation results saved to {output_path}")


/Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/week7/lib/python3.10/site-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


✅ Evaluation results saved to /Users/elbethelzewdie/Downloads/rag-complaint-chatbot/rag-complaint-chatbot/data/rag_evaluation_results.csv


In [8]:
# --- 5. Display DataFrame ---
df_evaluation = pd.DataFrame(results)
df_evaluation

,Question,Generated Answer,Retrieved Sources,Accuracy (1-5),Relevance (1-5),Completeness (1-5),Hallucination (Yes/No),Comments
0,Why do customers report being charged unexpect...,Customers report being charged unexpected fees...,"Fees or interest Bread Financial Holdings, Inc...",1,1,5,Yes,
1,What are the common complaints regarding credi...,"Based on the provided context, common complain...",Problem with a purchase shown on your statemen...,1,1,5,Yes,
2,Why are some personal loan applications denied...,Personal loan applications can be denied for s...,"Getting the loan CITIBANK, N.A. | Fraud or sca...",1,1,5,Yes,
3,What issues do customers report about personal...,Customers report issues related to personal lo...,Managing an account JPMORGAN CHASE & CO. | Cha...,2,2,5,Yes,
4,What problems do customers report with opening...,"Based on the provided context, the issues cust...","Closing an account BMO Bank, N.A. | Rewards BA...",1,1,5,Yes,
5,Are there complaints about delayed transaction...,"Based on the provided context, there is no dir...",Managing an account ALLY FINANCIAL INC. | Clos...,2,2,5,Yes,
6,What are the most frequent complaints customer...,"Based on the provided context, the most freque...",Managing an account U.S. BANCORP | Managing an...,1,1,5,Yes,
7,Why do customers report delays or failures whe...,Customers report delays or failures when makin...,Money was not available when promised BANK OF ...,1,1,5,Yes,
